In [1]:
import networkx as nx
import json as json

In [2]:
with open("Data/MMS_rv.json", 'r') as f:
    Data =  json.load(f)

In [3]:
type(Data)

list

In [4]:
Data[0]['ID']

'1192168487'

In [5]:
for doc in Data:
    if(doc['ID'] == "other"):
        del(doc)
    elif("other" in doc['ID'] or "unspecified" in doc['ID']):
        pos = doc['ID'].find("/")
        doc['ID'] = doc['ID'][:pos]
    else:
        continue

In [6]:
cardio_list = list()

In [7]:
def extract_cardio(cardio):
    for doc in Data:
        if(doc['ID'] == cardio):
            l = len(doc['childs'])
            if(l > 0):
                for i in range(l):
                    cardio_list.append(doc['childs'][i])
                    extract_cardio(doc['childs'][i])

In [8]:
for doc in Data:
    if(doc['ID'] == "426429380"):
        l = len(doc['childs'])
        if(l > 0):
            for i in range(l):
                cardio_list.append(doc['childs'][i])
                extract_cardio(doc['childs'][i])

In [9]:
print(cardio_list)

['539975693', '966120194', '924915526', '761947693', '1917449952', '1461918574', '1788452129', '761947693/other', '761947693/unspecified', '1210166201', '1015872326', '1002963291', '1331849426', '1663360295', '1713204963', '56009591', '1663360295/other', '1663360295/unspecified', '1964269418', '1829545371', '718946808', '253490233', '216789527', '718946808/other', '718946808/unspecified', '1334938734', '391388807', '680129526', '1334938734/unspecified', '993844371', '1401866267', '2070501946', '993844371/unspecified', '904209735', '1829545371/unspecified', '1221742343', '956002477', '1259725625', '1221742343/unspecified', '1584861447', '1353640147', '910277130', '456973650', '528140652', '318811736', '847963591', '165648238', '8597952', '1063181491', '147717581', '1584861447/other', '1584861447/unspecified', '1964269418/unspecified', '1059873720', '1280712786', '1911344636', '1399791513', '1694252775', '1280712786/unspecified', '777924269', '464855259', '2104517003', '933854832', '4980